Chunks generated by [MinerU](https://github.com/opendatalab/MinerU) are stored inside `output/{file_name}` . 
- This script will do some preprocessing and save the chunks to `output/{file_name}/data.csv` file

In [1]:
from dotenv import load_dotenv
import boto3
import os
import json,csv

load_dotenv()
s3_client = boto3.client(
        service_name="s3",
        region_name=os.getenv("AWS_DEFAULT_REGION"),
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    )

In [5]:
file_name = "tata_punch_owner_manual.pdf"

FOLDER_NAME = file_name.split(".")[0]
BUCKET = "b-public"
OBJECT_PREFIX = "document-extractor"

folder_path = f"output/{FOLDER_NAME}"
content_lst = f"{folder_path}/{FOLDER_NAME}_content_list.json"
csv_file = f"{folder_path}/{FOLDER_NAME}_csv.csv"


In [6]:
with open(content_lst,"r") as f:
    content_lst_obj = json.loads(f.read())

fin_cont = []
for content_lst in content_lst_obj:
    page_num = content_lst["page_idx"]+1
    if content_lst["type"] == "image" or content_lst["type"] == "table":
        content = ""
        s3_url = ""
        if "table_body" in content_lst:
            content = content_lst["table_body"]


        if "img_path" in content_lst and content_lst["img_path"]:    
            file_path = f"{folder_path}/{content_lst['img_path']}"
            object_path = file_path.replace("output",OBJECT_PREFIX)
            s3_url = f"https://{BUCKET}.s3.us-west-2.amazonaws.com/{object_path}"

            s3_client.upload_file(file_path, BUCKET, object_path)
        fin_cont.append([content,page_num,s3_url])        
    elif content_lst["type"] == "text":
        txt = content_lst["text"]
        if "text_level" in content_lst:
            txt=("#"*content_lst["text_level"])+txt
        fin_cont.append([txt,page_num,""])

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(fin_cont)
        